#### 고려대한국어대사전 오픈프로 포멧으로 변경 (idiom)

1. 오픈프로 필요 기타 칼럼 추가
2. 표제어 자리에 들어갈 부분 추출 : < > 제거 
3. 표제어의 설명부분 숫자별로 분류 (분류해서 새로운 행을 생성해야함)
4. 설명 부분의 < > 부분 용법 내용으로 분류 
5. 품사 폰트 변경
6. "¶" 이걸 기준으로 설명과 예시 분류
7. 용례 유의어 따로 정리 ex> <유> 탈(을) 쓰다.
8. 용례 여러개일 경우 분류
9. <> [] 제거 만들기  
10. 뜻풀이 참조어 링크 태그로 변환
11. 중복제거
12. 네이버와 아이디 매칭

In [68]:
import pandas as pd
import re

pattern_format = {'ꄍ' : '(명사)', 'ꄉ' : '(동사)', 'ꄺ' : '(형용사)', 'ꄖ' : '(부사)'}
symbols = ['①', '②', '③']

# input_file_path 부분을 채워주세요.
input_file_path = "/Users/anjonghyeon/Desktop/KU/민족문화연구원 사전실/openpro/kordic/asset/idiom.csv"
target = pd.read_csv(input_file_path)
target

,uid,eid,ohw,ocol,idiom,explan
0,1,443,가꾸로,가꾸로박히다,◁가꾸로 박히다▷,◀ꄍ이 ꄍ에▶ (사람이 바닥에) 머리가 땅을 향한 채로 떨어지다. ¶아이가 침대에서...
1,2,459,가난,가난이들다,◁가난이 들다▷,①◀ꄍ에▶ (집이나 나라에) 어려움이 닥쳐 살림살이가 넉넉하지 못하게 되다. ¶부잣...
2,4,583,가늠,가늠을보다,◁가늠(을) 보다〔잡다〕▷,①◀ꄍ이 ꄍ을▶ (사람이 목표나 형편을) 겨누어 보다. ¶강 이사가 이번 일의 성패...
3,5,611,가다,간곳이없다,◁간 곳(이) 없다 / 간 데(가) 없다▷,◀ꄍ이▶ (무엇이) 완전히 없어지다. ¶무분별한 아파트 건설로 고향땅의 녹지들이 간...
4,7,627,가닥,가닥을잡다,◁가닥을 잡다▷,①◀ꄍ이 ꄍ으로▶ (사건이나 문제가 해결 방향으로) 실마리를 얻다. ¶폭력 사건이 ...
...,...,...,...,...,...,...
3757,5276,267972,입,입이궁금하다,◁입이 궁금하다▷,"◀ꄍ이▶ (사람이) 무엇을 먹고 싶은 생각이 일어나다. ¶입이 궁금한데, 우리 고구..."
3758,5277,69605,눈치,눈치로때려잡다,◁눈치로 때려잡다▷,◀ꄍ이 ꄍ을▶ (사람이 상황이나 분위기를) 대충 미루어 짐작하다. ¶나는 회사 돌아...
3759,5278,185641,손꼽다,손꼽아기다리다,◁손꼽아 기다리다▷,◀ꄍ이 ꄍ을▶ (사람이 어떤 날이나 일을) 초조한 마음으로 기다리다. ¶철수는 제대...
3760,5279,69525,눈앞,눈앞이노래지다,◁눈앞이 노래지다〔누레지다〕▷,①◀ꄍ이▶ (사람이) 기력이 쇠하여 정신이 아찔하게 되다. ¶나는 수술 후유증이 남...


In [69]:
# 1. 오픈프로 필요한 기타 칼럼 추가 
target['type'] = '숙어'
target['typediff'] = '괄호 없앤 속담/관용구'
target['onum'] = int(1)
target = target[['uid', 'eid', 'ohw', 'ocol', 'idiom', 'type', 'typediff', 'explan', 'onum']]
target

,uid,eid,ohw,ocol,idiom,type,typediff,explan,onum
0,1,443,가꾸로,가꾸로박히다,◁가꾸로 박히다▷,숙어,괄호 없앤 속담/관용구,◀ꄍ이 ꄍ에▶ (사람이 바닥에) 머리가 땅을 향한 채로 떨어지다. ¶아이가 침대에서...,1
1,2,459,가난,가난이들다,◁가난이 들다▷,숙어,괄호 없앤 속담/관용구,①◀ꄍ에▶ (집이나 나라에) 어려움이 닥쳐 살림살이가 넉넉하지 못하게 되다. ¶부잣...,1
2,4,583,가늠,가늠을보다,◁가늠(을) 보다〔잡다〕▷,숙어,괄호 없앤 속담/관용구,①◀ꄍ이 ꄍ을▶ (사람이 목표나 형편을) 겨누어 보다. ¶강 이사가 이번 일의 성패...,1
3,5,611,가다,간곳이없다,◁간 곳(이) 없다 / 간 데(가) 없다▷,숙어,괄호 없앤 속담/관용구,◀ꄍ이▶ (무엇이) 완전히 없어지다. ¶무분별한 아파트 건설로 고향땅의 녹지들이 간...,1
4,7,627,가닥,가닥을잡다,◁가닥을 잡다▷,숙어,괄호 없앤 속담/관용구,①◀ꄍ이 ꄍ으로▶ (사건이나 문제가 해결 방향으로) 실마리를 얻다. ¶폭력 사건이 ...,1
...,...,...,...,...,...,...,...,...,...
3757,5276,267972,입,입이궁금하다,◁입이 궁금하다▷,숙어,괄호 없앤 속담/관용구,"◀ꄍ이▶ (사람이) 무엇을 먹고 싶은 생각이 일어나다. ¶입이 궁금한데, 우리 고구...",1
3758,5277,69605,눈치,눈치로때려잡다,◁눈치로 때려잡다▷,숙어,괄호 없앤 속담/관용구,◀ꄍ이 ꄍ을▶ (사람이 상황이나 분위기를) 대충 미루어 짐작하다. ¶나는 회사 돌아...,1
3759,5278,185641,손꼽다,손꼽아기다리다,◁손꼽아 기다리다▷,숙어,괄호 없앤 속담/관용구,◀ꄍ이 ꄍ을▶ (사람이 어떤 날이나 일을) 초조한 마음으로 기다리다. ¶철수는 제대...,1
3760,5279,69525,눈앞,눈앞이노래지다,◁눈앞이 노래지다〔누레지다〕▷,숙어,괄호 없앤 속담/관용구,①◀ꄍ이▶ (사람이) 기력이 쇠하여 정신이 아찔하게 되다. ¶나는 수술 후유증이 남...,1


In [70]:
# 2. 표제어 자리에 들어갈 부분 추출 : < > 제거 
LEN = len(target)
for i in range(LEN):
    target.loc[i, 'idiom'] = target.loc[i, 'idiom'][1:-1]
target

,uid,eid,ohw,ocol,idiom,type,typediff,explan,onum
0,1,443,가꾸로,가꾸로박히다,가꾸로 박히다,숙어,괄호 없앤 속담/관용구,◀ꄍ이 ꄍ에▶ (사람이 바닥에) 머리가 땅을 향한 채로 떨어지다. ¶아이가 침대에서...,1
1,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,①◀ꄍ에▶ (집이나 나라에) 어려움이 닥쳐 살림살이가 넉넉하지 못하게 되다. ¶부잣...,1
2,4,583,가늠,가늠을보다,가늠(을) 보다〔잡다〕,숙어,괄호 없앤 속담/관용구,①◀ꄍ이 ꄍ을▶ (사람이 목표나 형편을) 겨누어 보다. ¶강 이사가 이번 일의 성패...,1
3,5,611,가다,간곳이없다,간 곳(이) 없다 / 간 데(가) 없다,숙어,괄호 없앤 속담/관용구,◀ꄍ이▶ (무엇이) 완전히 없어지다. ¶무분별한 아파트 건설로 고향땅의 녹지들이 간...,1
4,7,627,가닥,가닥을잡다,가닥을 잡다,숙어,괄호 없앤 속담/관용구,①◀ꄍ이 ꄍ으로▶ (사건이나 문제가 해결 방향으로) 실마리를 얻다. ¶폭력 사건이 ...,1
...,...,...,...,...,...,...,...,...,...
3757,5276,267972,입,입이궁금하다,입이 궁금하다,숙어,괄호 없앤 속담/관용구,"◀ꄍ이▶ (사람이) 무엇을 먹고 싶은 생각이 일어나다. ¶입이 궁금한데, 우리 고구...",1
3758,5277,69605,눈치,눈치로때려잡다,눈치로 때려잡다,숙어,괄호 없앤 속담/관용구,◀ꄍ이 ꄍ을▶ (사람이 상황이나 분위기를) 대충 미루어 짐작하다. ¶나는 회사 돌아...,1
3759,5278,185641,손꼽다,손꼽아기다리다,손꼽아 기다리다,숙어,괄호 없앤 속담/관용구,◀ꄍ이 ꄍ을▶ (사람이 어떤 날이나 일을) 초조한 마음으로 기다리다. ¶철수는 제대...,1
3760,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,①◀ꄍ이▶ (사람이) 기력이 쇠하여 정신이 아찔하게 되다. ¶나는 수술 후유증이 남...,1


In [71]:
# 3. 표제어의 설명부분 숫자별로 분류 (분류해서 새로운 행을 생성해야함)
extra_list = []
LEN = len(target)
for i in range(LEN):
    if symbols[0] in target.loc[i, 'explan']:
        temps = target.loc[i, 'explan'].split(symbols[1])
        target.loc[i, 'explan'] = temps[0][1:].strip()
        new_row = target.loc[i, :].copy()
        new_row['onum'] = 2
        new_row['explan'] = temps[1].strip()
        extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'idiom':[extra_list[i]['idiom']], 'type':[extra_list[i]['type']], 'typediff':[extra_list[i]['typediff']], 
                            'explan':[extra_list[i]['explan']], 'onum':[extra_list[i]['onum']]})
    target = pd.concat([target, new_row], ignore_index=True)

target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

extra_list = []
LEN = len(target)
for i in range(LEN):
    if symbols[2] in target.loc[i, 'explan']:
        temps = target.loc[i, 'explan'].split(symbols[2])
        target.loc[i, 'explan'] = temps[0].strip()
        new_row = target.loc[i, :].copy()
        new_row['explan'] = temps[1].strip()
        new_row['onum'] = 3
        extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'idiom':[extra_list[i]['idiom']], 'type':[extra_list[i]['type']], 'typediff':[extra_list[i]['typediff']], 
                            'explan':[extra_list[i]['explan']], 'onum':[extra_list[i]['onum']]})
    target = pd.concat([target, new_row], ignore_index=True)

target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)
target

,uid,eid,ohw,ocol,idiom,type,typediff,explan,onum
0,1,443,가꾸로,가꾸로박히다,가꾸로 박히다,숙어,괄호 없앤 속담/관용구,◀ꄍ이 ꄍ에▶ (사람이 바닥에) 머리가 땅을 향한 채로 떨어지다. ¶아이가 침대에서...,1
1,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,◀ꄍ에▶ (집이나 나라에) 어려움이 닥쳐 살림살이가 넉넉하지 못하게 되다. ¶부잣집...,1
2,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,◀ꄍ이▶ (무엇이) 드물어 구하기 어렵게 되다. ¶요즘은 인재가 가난이 들었어요.,2
3,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,◀ꄍ이▶ (농작물이) 여느 해보다 아주 적게 수확되다. ¶비가 제때 안 와서 배추가...,3
4,4,583,가늠,가늠을보다,가늠(을) 보다〔잡다〕,숙어,괄호 없앤 속담/관용구,◀ꄍ이 ꄍ을▶ (사람이 목표나 형편을) 겨누어 보다. ¶강 이사가 이번 일의 성패를...,1
...,...,...,...,...,...,...,...,...,...
4377,5277,69605,눈치,눈치로때려잡다,눈치로 때려잡다,숙어,괄호 없앤 속담/관용구,◀ꄍ이 ꄍ을▶ (사람이 상황이나 분위기를) 대충 미루어 짐작하다. ¶나는 회사 돌아...,1
4378,5278,185641,손꼽다,손꼽아기다리다,손꼽아 기다리다,숙어,괄호 없앤 속담/관용구,◀ꄍ이 ꄍ을▶ (사람이 어떤 날이나 일을) 초조한 마음으로 기다리다. ¶철수는 제대...,1
4379,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,◀ꄍ이▶ (사람이) 기력이 쇠하여 정신이 아찔하게 되다. ¶나는 수술 후유증이 남아...,1
4380,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,◀ꄍ이▶ (사람이) 지나치게 상심하여 어찌할 바를 모르다. ¶그때 우리는 아버지의 ...,2


In [72]:
# 4. 설명 부분의 < > 부분 용법 내용으로 분류 
target['pattern'] = ''

LEN = len(target)
for i in range(LEN):
    p = r'\s*◀(.*?)▶' 
    m = re.search(p, target.loc[i, 'explan'])
    if m:
        result = m.group().strip() 
        target.loc[i, 'pattern'] = result[1:-1] 
        target.loc[i, 'explan'] = re.sub(p, '', target.loc[i, 'explan']).strip() 
target

,uid,eid,ohw,ocol,idiom,type,typediff,explan,onum,pattern
0,1,443,가꾸로,가꾸로박히다,가꾸로 박히다,숙어,괄호 없앤 속담/관용구,(사람이 바닥에) 머리가 땅을 향한 채로 떨어지다. ¶아이가 침대에서 장난을 치다가...,1,ꄍ이 ꄍ에
1,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,(집이나 나라에) 어려움이 닥쳐 살림살이가 넉넉하지 못하게 되다. ¶부잣집에 가난이...,1,ꄍ에
2,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,(무엇이) 드물어 구하기 어렵게 되다. ¶요즘은 인재가 가난이 들었어요.,2,ꄍ이
3,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,(농작물이) 여느 해보다 아주 적게 수확되다. ¶비가 제때 안 와서 배추가 가난이 ...,3,ꄍ이
4,4,583,가늠,가늠을보다,가늠(을) 보다〔잡다〕,숙어,괄호 없앤 속담/관용구,(사람이 목표나 형편을) 겨누어 보다. ¶강 이사가 이번 일의 성패를 가늠을 봤지만...,1,ꄍ이 ꄍ을
...,...,...,...,...,...,...,...,...,...,...
4377,5277,69605,눈치,눈치로때려잡다,눈치로 때려잡다,숙어,괄호 없앤 속담/관용구,(사람이 상황이나 분위기를) 대충 미루어 짐작하다. ¶나는 회사 돌아가는 상황이 심...,1,ꄍ이 ꄍ을
4378,5278,185641,손꼽다,손꼽아기다리다,손꼽아 기다리다,숙어,괄호 없앤 속담/관용구,(사람이 어떤 날이나 일을) 초조한 마음으로 기다리다. ¶철수는 제대하는 날만을 손...,1,ꄍ이 ꄍ을
4379,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,(사람이) 기력이 쇠하여 정신이 아찔하게 되다. ¶나는 수술 후유증이 남아서 조금만...,1,ꄍ이
4380,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,(사람이) 지나치게 상심하여 어찌할 바를 모르다. ¶그때 우리는 아버지의 사고 소식...,2,ꄍ이


In [73]:
# 5. 품사 폰트 변경
LEN = len(target)
for i in range(LEN):
    if target.loc[i, 'pattern'] == '':
        continue
    for pattern in pattern_format:
        target.loc[i, 'pattern'] = re.sub(pattern, pattern_format[pattern], target.loc[i, 'pattern'])

target = target[['uid', 'eid', 'ohw', 'ocol', 'idiom', 'type', 'typediff', 'onum', 'pattern', 'explan']]
target

,uid,eid,ohw,ocol,idiom,type,typediff,onum,pattern,explan
0,1,443,가꾸로,가꾸로박히다,가꾸로 박히다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)에,(사람이 바닥에) 머리가 땅을 향한 채로 떨어지다. ¶아이가 침대에서 장난을 치다가...
1,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,1,(명사)에,(집이나 나라에) 어려움이 닥쳐 살림살이가 넉넉하지 못하게 되다. ¶부잣집에 가난이...
2,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,2,(명사)이,(무엇이) 드물어 구하기 어렵게 되다. ¶요즘은 인재가 가난이 들었어요.
3,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,3,(명사)이,(농작물이) 여느 해보다 아주 적게 수확되다. ¶비가 제때 안 와서 배추가 가난이 ...
4,4,583,가늠,가늠을보다,가늠(을) 보다〔잡다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 목표나 형편을) 겨누어 보다. ¶강 이사가 이번 일의 성패를 가늠을 봤지만...
...,...,...,...,...,...,...,...,...,...,...
4377,5277,69605,눈치,눈치로때려잡다,눈치로 때려잡다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 상황이나 분위기를) 대충 미루어 짐작하다. ¶나는 회사 돌아가는 상황이 심...
4378,5278,185641,손꼽다,손꼽아기다리다,손꼽아 기다리다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 어떤 날이나 일을) 초조한 마음으로 기다리다. ¶철수는 제대하는 날만을 손...
4379,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이,(사람이) 기력이 쇠하여 정신이 아찔하게 되다. ¶나는 수술 후유증이 남아서 조금만...
4380,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,2,(명사)이,(사람이) 지나치게 상심하여 어찌할 바를 모르다. ¶그때 우리는 아버지의 사고 소식...


In [74]:
# 6. "¶" 이걸 기준으로 설명과 예시 분류
LEN = len(target)
target['explan'] = target['explan'].str.strip()
for i in range(LEN):
    temp = target.loc[i, 'explan'].split('¶')
    target.loc[i, 'explan'] = temp[0].strip()
    if len(temp) != 1:
        target.loc[i, 'example'] = temp[1].strip()
target

,uid,eid,ohw,ocol,idiom,type,typediff,onum,pattern,explan,example
0,1,443,가꾸로,가꾸로박히다,가꾸로 박히다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)에,(사람이 바닥에) 머리가 땅을 향한 채로 떨어지다.,아이가 침대에서 장난을 치다가 방바닥에 가꾸로 박혔다.
1,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,1,(명사)에,(집이나 나라에) 어려움이 닥쳐 살림살이가 넉넉하지 못하게 되다.,부잣집에 가난이 드니 전과 달리 인심이 박해졌다.
2,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,2,(명사)이,(무엇이) 드물어 구하기 어렵게 되다.,요즘은 인재가 가난이 들었어요.
3,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,3,(명사)이,(농작물이) 여느 해보다 아주 적게 수확되다.,비가 제때 안 와서 배추가 가난이 들었다.
4,4,583,가늠,가늠을보다,가늠(을) 보다〔잡다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 목표나 형편을) 겨누어 보다.,강 이사가 이번 일의 성패를 가늠을 봤지만 결과를 정확히 예측할 수는 없었다.
...,...,...,...,...,...,...,...,...,...,...,...
4377,5277,69605,눈치,눈치로때려잡다,눈치로 때려잡다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 상황이나 분위기를) 대충 미루어 짐작하다.,나는 회사 돌아가는 상황이 심상치 않음을 눈치로 때려잡았다. / 막내는 뭔가 자기만...
4378,5278,185641,손꼽다,손꼽아기다리다,손꼽아 기다리다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 어떤 날이나 일을) 초조한 마음으로 기다리다.,철수는 제대하는 날만을 손꼽아 기다렸다. / 나는 어머니와 같이 외출하는 날만을 손...
4379,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이,(사람이) 기력이 쇠하여 정신이 아찔하게 되다.,나는 수술 후유증이 남아서 조금만 무리를 해도 눈앞이 노래진다.
4380,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,2,(명사)이,(사람이) 지나치게 상심하여 어찌할 바를 모르다.,그때 우리는 아버지의 사고 소식을 듣고 눈앞이 노래지는 것 같았다.


In [75]:
# 7. 용례 유의어 따로 정리 ex> <유> 탈(을) 쓰다. (뜻번호당 하나만!!! 용례당 하나가 아님)
target['example'].fillna('', inplace=True)
target['similar_expression'] = ''

LEN = len(target)
for i in range(LEN):
    if '<유>' in target.loc[i, 'example']: 
        temps = target.loc[i, 'example'].split('<유>')
        target.loc[i, 'example'] = temps[0]
        target.loc[i, 'similar_expression'] = temps[1].strip() 
target

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_17808/3899923710.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  target['example'].fillna('', inplace=True)


,uid,eid,ohw,ocol,idiom,type,typediff,onum,pattern,explan,example,similar_expression
0,1,443,가꾸로,가꾸로박히다,가꾸로 박히다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)에,(사람이 바닥에) 머리가 땅을 향한 채로 떨어지다.,아이가 침대에서 장난을 치다가 방바닥에 가꾸로 박혔다.,
1,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,1,(명사)에,(집이나 나라에) 어려움이 닥쳐 살림살이가 넉넉하지 못하게 되다.,부잣집에 가난이 드니 전과 달리 인심이 박해졌다.,
2,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,2,(명사)이,(무엇이) 드물어 구하기 어렵게 되다.,요즘은 인재가 가난이 들었어요.,
3,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,3,(명사)이,(농작물이) 여느 해보다 아주 적게 수확되다.,비가 제때 안 와서 배추가 가난이 들었다.,
4,4,583,가늠,가늠을보다,가늠(을) 보다〔잡다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 목표나 형편을) 겨누어 보다.,강 이사가 이번 일의 성패를 가늠을 봤지만 결과를 정확히 예측할 수는 없었다.,
...,...,...,...,...,...,...,...,...,...,...,...,...
4377,5277,69605,눈치,눈치로때려잡다,눈치로 때려잡다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 상황이나 분위기를) 대충 미루어 짐작하다.,나는 회사 돌아가는 상황이 심상치 않음을 눈치로 때려잡았다. / 막내는 뭔가 자기만...,
4378,5278,185641,손꼽다,손꼽아기다리다,손꼽아 기다리다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 어떤 날이나 일을) 초조한 마음으로 기다리다.,철수는 제대하는 날만을 손꼽아 기다렸다. / 나는 어머니와 같이 외출하는 날만을 손...,
4379,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이,(사람이) 기력이 쇠하여 정신이 아찔하게 되다.,나는 수술 후유증이 남아서 조금만 무리를 해도 눈앞이 노래진다.,
4380,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,2,(명사)이,(사람이) 지나치게 상심하여 어찌할 바를 모르다.,그때 우리는 아버지의 사고 소식을 듣고 눈앞이 노래지는 것 같았다.,


In [76]:
# 8. 용례 여러개일 경우 분류
LEN = len(target)
extra_list = []
for i in range(LEN):
    if '/' in target.loc[i, 'example']:
        temps = target.loc[i, 'example'].split(' / ')
        target.loc[i, 'example'] = temps[0].strip()
        for example in temps[1:]:
            new_row = target.loc[i, :].copy()
            new_row['example'] = temps[1].strip()
            extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'idiom':[extra_list[i]['idiom']], 'type':[extra_list[i]['type']], 'typediff':[extra_list[i]['typediff']], 
                            'onum':[extra_list[i]['onum']], 'pattern':[extra_list[i]['pattern']], 'explan':[extra_list[i]['explan']],
                            'example':extra_list[i]['example']})
    target = pd.concat([target, new_row], ignore_index=True)

target['similar_expression'].fillna('', inplace=True)
target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)
target

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_17808/3324017618.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  target['similar_expression'].fillna('', inplace=True)


,uid,eid,ohw,ocol,idiom,type,typediff,onum,pattern,explan,example,similar_expression
0,1,443,가꾸로,가꾸로박히다,가꾸로 박히다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)에,(사람이 바닥에) 머리가 땅을 향한 채로 떨어지다.,아이가 침대에서 장난을 치다가 방바닥에 가꾸로 박혔다.,
1,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,1,(명사)에,(집이나 나라에) 어려움이 닥쳐 살림살이가 넉넉하지 못하게 되다.,부잣집에 가난이 드니 전과 달리 인심이 박해졌다.,
2,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,2,(명사)이,(무엇이) 드물어 구하기 어렵게 되다.,요즘은 인재가 가난이 들었어요.,
3,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,3,(명사)이,(농작물이) 여느 해보다 아주 적게 수확되다.,비가 제때 안 와서 배추가 가난이 들었다.,
4,4,583,가늠,가늠을보다,가늠(을) 보다〔잡다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 목표나 형편을) 겨누어 보다.,강 이사가 이번 일의 성패를 가늠을 봤지만 결과를 정확히 예측할 수는 없었다.,
...,...,...,...,...,...,...,...,...,...,...,...,...
5602,5278,185641,손꼽다,손꼽아기다리다,손꼽아 기다리다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 어떤 날이나 일을) 초조한 마음으로 기다리다.,철수는 제대하는 날만을 손꼽아 기다렸다.,
5603,5278,185641,손꼽다,손꼽아기다리다,손꼽아 기다리다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 어떤 날이나 일을) 초조한 마음으로 기다리다.,나는 어머니와 같이 외출하는 날만을 손꼽아 기다렸다.,
5604,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이,(사람이) 기력이 쇠하여 정신이 아찔하게 되다.,나는 수술 후유증이 남아서 조금만 무리를 해도 눈앞이 노래진다.,
5605,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,2,(명사)이,(사람이) 지나치게 상심하여 어찌할 바를 모르다.,그때 우리는 아버지의 사고 소식을 듣고 눈앞이 노래지는 것 같았다.,


In [77]:
target

,uid,eid,ohw,ocol,idiom,type,typediff,onum,pattern,explan,example,similar_expression
0,1,443,가꾸로,가꾸로박히다,가꾸로 박히다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)에,(사람이 바닥에) 머리가 땅을 향한 채로 떨어지다.,아이가 침대에서 장난을 치다가 방바닥에 가꾸로 박혔다.,
1,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,1,(명사)에,(집이나 나라에) 어려움이 닥쳐 살림살이가 넉넉하지 못하게 되다.,부잣집에 가난이 드니 전과 달리 인심이 박해졌다.,
2,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,2,(명사)이,(무엇이) 드물어 구하기 어렵게 되다.,요즘은 인재가 가난이 들었어요.,
3,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,3,(명사)이,(농작물이) 여느 해보다 아주 적게 수확되다.,비가 제때 안 와서 배추가 가난이 들었다.,
4,4,583,가늠,가늠을보다,가늠(을) 보다〔잡다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 목표나 형편을) 겨누어 보다.,강 이사가 이번 일의 성패를 가늠을 봤지만 결과를 정확히 예측할 수는 없었다.,
...,...,...,...,...,...,...,...,...,...,...,...,...
5602,5278,185641,손꼽다,손꼽아기다리다,손꼽아 기다리다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 어떤 날이나 일을) 초조한 마음으로 기다리다.,철수는 제대하는 날만을 손꼽아 기다렸다.,
5603,5278,185641,손꼽다,손꼽아기다리다,손꼽아 기다리다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 어떤 날이나 일을) 초조한 마음으로 기다리다.,나는 어머니와 같이 외출하는 날만을 손꼽아 기다렸다.,
5604,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이,(사람이) 기력이 쇠하여 정신이 아찔하게 되다.,나는 수술 후유증이 남아서 조금만 무리를 해도 눈앞이 노래진다.,
5605,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,2,(명사)이,(사람이) 지나치게 상심하여 어찌할 바를 모르다.,그때 우리는 아버지의 사고 소식을 듣고 눈앞이 노래지는 것 같았다.,


In [78]:
# 10. 뜻풀이 참조어 링크 태그로 변환

In [79]:
# 11. 중복제거
condition = target.duplicated(subset=['uid', 'onum'], keep='first')
condition2 = target.duplicated(subset=['uid'], keep='first')
target.loc[condition, 'uid':'explan'] = ""
target.loc[condition2, 'uid':'typediff'] = ""
target

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_17808/3298006694.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.loc[condition, 'uid':'explan'] = ""
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_17808/3298006694.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.loc[condition, 'uid':'explan'] = ""
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_17808/3298006694.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.l

,uid,eid,ohw,ocol,idiom,type,typediff,onum,pattern,explan,example,similar_expression
0,1,443,가꾸로,가꾸로박히다,가꾸로 박히다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)에,(사람이 바닥에) 머리가 땅을 향한 채로 떨어지다.,아이가 침대에서 장난을 치다가 방바닥에 가꾸로 박혔다.,
1,2,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,1,(명사)에,(집이나 나라에) 어려움이 닥쳐 살림살이가 넉넉하지 못하게 되다.,부잣집에 가난이 드니 전과 달리 인심이 박해졌다.,
2,,,,,,,,2,(명사)이,(무엇이) 드물어 구하기 어렵게 되다.,요즘은 인재가 가난이 들었어요.,
3,,,,,,,,3,(명사)이,(농작물이) 여느 해보다 아주 적게 수확되다.,비가 제때 안 와서 배추가 가난이 들었다.,
4,4,583,가늠,가늠을보다,가늠(을) 보다〔잡다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 목표나 형편을) 겨누어 보다.,강 이사가 이번 일의 성패를 가늠을 봤지만 결과를 정확히 예측할 수는 없었다.,
...,...,...,...,...,...,...,...,...,...,...,...,...
5602,5278,185641,손꼽다,손꼽아기다리다,손꼽아 기다리다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 어떤 날이나 일을) 초조한 마음으로 기다리다.,철수는 제대하는 날만을 손꼽아 기다렸다.,
5603,,,,,,,,,,,나는 어머니와 같이 외출하는 날만을 손꼽아 기다렸다.,
5604,5279,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이,(사람이) 기력이 쇠하여 정신이 아찔하게 되다.,나는 수술 후유증이 남아서 조금만 무리를 해도 눈앞이 노래진다.,
5605,,,,,,,,2,(명사)이,(사람이) 지나치게 상심하여 어찌할 바를 모르다.,그때 우리는 아버지의 사고 소식을 듣고 눈앞이 노래지는 것 같았다.,


In [80]:
ID

,naver_id,type,idiom,shoulder
0,4a81f70b9f6046fe908cc538c9940a68,숙어,가꾸로 박히다,0
1,032b6d148bd6465caf7ebf5789028e31,숙어,가난이 들다,0
2,cfb56075b9984f8e951f7131a2a1229e,숙어,가늠(을) 보다〔잡다〕,0
3,735e02b0d5dd4abd9ea3eb372fb7a3da,숙어,가늠이 가다,0
4,c719e1afdc974f209061fc9146b952ed,숙어,가닥을 잡다,0
...,...,...,...,...
3774,04b68b0042fb4f008621a70df1856a3e,숙어,흰 눈으로 보다,0
3775,99e89c318efd4b65ab20099a2d044869,숙어,흰소리(를) 치다,0
3776,f3f97434863a463fbc245705216c996f,숙어,히에기가 재다,0
3777,30c476555cff40cbacba5351bf356df4,숙어,힘을 올리다,0


In [81]:
# 12. 네이버와 아이디 매칭
target['naver_id'] = ""
target = target[['uid', 'naver_id', 'eid', 'ohw', 'ocol', 'idiom', 'type', 'typediff', 'onum',
        'pattern', 'explan', 'example', 'similar_expression']]

ID = pd.read_csv('../asset/idiom_id.csv')
LEN = len(target)
LEN_ID = len(ID)
for i in range(LEN):
        target_idiom = target.loc[i, 'idiom']
        start = 0
        end = LEN_ID - 1
        while True:
                if start > end:
                        break
                mid = (start + end) // 2
                try_idiom = ID.loc[mid, 'idiom']
                if target_idiom == try_idiom:
                        target.loc[i, 'naver_id'] = ID.loc[mid, 'naver_id']
                        break
                elif target_idiom < try_idiom:
                        end = mid - 1
                else:
                        start = mid + 1

target

,uid,naver_id,eid,ohw,ocol,idiom,type,typediff,onum,pattern,explan,example,similar_expression
0,1,4a81f70b9f6046fe908cc538c9940a68,443,가꾸로,가꾸로박히다,가꾸로 박히다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)에,(사람이 바닥에) 머리가 땅을 향한 채로 떨어지다.,아이가 침대에서 장난을 치다가 방바닥에 가꾸로 박혔다.,
1,2,032b6d148bd6465caf7ebf5789028e31,459,가난,가난이들다,가난이 들다,숙어,괄호 없앤 속담/관용구,1,(명사)에,(집이나 나라에) 어려움이 닥쳐 살림살이가 넉넉하지 못하게 되다.,부잣집에 가난이 드니 전과 달리 인심이 박해졌다.,
2,,,,,,,,,2,(명사)이,(무엇이) 드물어 구하기 어렵게 되다.,요즘은 인재가 가난이 들었어요.,
3,,,,,,,,,3,(명사)이,(농작물이) 여느 해보다 아주 적게 수확되다.,비가 제때 안 와서 배추가 가난이 들었다.,
4,4,cfb56075b9984f8e951f7131a2a1229e,583,가늠,가늠을보다,가늠(을) 보다〔잡다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 목표나 형편을) 겨누어 보다.,강 이사가 이번 일의 성패를 가늠을 봤지만 결과를 정확히 예측할 수는 없었다.,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5602,5278,,185641,손꼽다,손꼽아기다리다,손꼽아 기다리다,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 어떤 날이나 일을) 초조한 마음으로 기다리다.,철수는 제대하는 날만을 손꼽아 기다렸다.,
5603,,,,,,,,,,,,나는 어머니와 같이 외출하는 날만을 손꼽아 기다렸다.,
5604,5279,,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,숙어,괄호 없앤 속담/관용구,1,(명사)이,(사람이) 기력이 쇠하여 정신이 아찔하게 되다.,나는 수술 후유증이 남아서 조금만 무리를 해도 눈앞이 노래진다.,
5605,,,,,,,,,2,(명사)이,(사람이) 지나치게 상심하여 어찌할 바를 모르다.,그때 우리는 아버지의 사고 소식을 듣고 눈앞이 노래지는 것 같았다.,


In [90]:
condition = (target['naver_id'] == "") & (target['uid'] != "")
# 신규어 개수 (매칭 안된 idiom)
len(target[condition])

43

In [91]:
target.to_csv('../result/processed_idiom.csv', index=False)
print("!!!FINISHED!!!")

!!!FINISHED!!!
